# Importing packages

In [24]:

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
print(tf.__version__)

# IMG_SHAPE = 192
# IMG_SHAPE = 224
IMG_SHAPE = 512

2.8.0


# Resizing the images

In [26]:
# load every image in the food folder and resize them to 224x224, then save them in a new folder

# for filename in os.listdir('food'):
#     img = Image.open('food/' + filename)
#     img = img.resize((IMG_SHAPE, IMG_SHAPE))
#     img.save('food_'+str(IMG_SHAPE)+'/' + filename)


# Extracting the features

### IMPORTANT: the code needs to be run 3 times with the right image size, to generate features with all 3 extractors. then combine them using the last cell

In [27]:


#efficientnetXL (needs 512x512)
# feature_extractor = hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2", input_shape=(IMG_SHAPE, IMG_SHAPE,3),trainable=False)

#food extractor (needs 192x192)
# feature_extractor = hub.KerasLayer("https://tfhub.dev/google/aiy/vision/classifier/food_V1/1", input_shape=(IMG_SHAPE, IMG_SHAPE,3),trainable=False)

#food extractor with resnet (needs 512x512)
feature_extractor = hub.KerasLayer("https://tfhub.dev/google/experts/bit/r50x1/in21k/food/1", input_shape=(IMG_SHAPE, IMG_SHAPE,3),trainable=False)


model = tf.keras.Sequential([
  feature_extractor,
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23500352  
                                                                 
Total params: 23,500,352
Trainable params: 0
Non-trainable params: 23,500,352
_________________________________________________________________


In [31]:
img = np.array(Image.open('food_'+str(IMG_SHAPE)+'/00001.jpg'))[None]*1.0/255
print(img.shape)
output = model.predict(img)
# print(output.shape)
print(output)
# print(img[0])

(1, 512, 512, 3)
[[0.02066039 0.         0.4547058  ... 0.34155965 1.0545528  0.26238483]]


In [33]:

for filename in os.listdir('food_'+str(IMG_SHAPE)+''):
    name = filename.replace(".jpg","")
    if(name != "02463"):
        continue

    img = Image.open('food_'+str(IMG_SHAPE)+'/' + filename)
    img = np.array(img)*1.0/255
    feature = model.predict(img[None])[0]
    print(np.max(feature))


25.08255


In [37]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_'+str(IMG_SHAPE)+''):
    line_id += 1
    img = Image.open('food_'+str(IMG_SHAPE)+'/' + filename)
    img = np.array(img)*1.0/255
    feature = model.predict(img[None])[0]
    features[filename.replace(".jpg","")] = feature
    if line_id % 100 == 0:
        print(line_id)
    # break

# save the features dictionary to a file using pickle
with open('features_resnet.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [38]:

# features = {}
# with open('features_mn.pickle', 'rb') as handle:
#     features = pickle.load(handle)
    
print(features['02461'])


[3.2279101e-01 1.6950681e-03 2.0503240e+00 ... 1.2365297e+00 1.3910511e+00
 9.2304796e-01]


In [40]:
print(len(features))
print(len(features["00000"]))

10000
2048


In [44]:
features1 = {}
features2 = {}
features3 = {}
with open('features_xl.pickle', 'rb') as handle:
    features1 = pickle.load(handle)
    print(len(features1["00000"]))
with open('features_mn.pickle', 'rb') as handle:
    features2 = pickle.load(handle)
    print(len(features2["00000"]))
with open('features_resnet.pickle', 'rb') as handle:
    features3 = pickle.load(handle)
    print(len(features3["00000"]))

features_merged = {}
for key in features1:
    features_merged[key] = np.concatenate((features1[key],features2[key],features3[key]))

print(len(features_merged))
print(len(features_merged["00000"]))

with open('features_merged.pickle', 'wb') as handle:
    pickle.dump(features_merged, handle, protocol=pickle.HIGHEST_PROTOCOL)

1280
2024
2048
10000
5352
